### Training a Neural Network Classifier: Step-by-Step

To train a neural network classifier, we follow these steps:

1. **Load Cleaned Data**: Start by loading the cleaned dataset.

2. **Define Hyperparameters and Data Splits**: Set the hyperparameters (e.g., learning rate, batch size) and split the data into training and testing sets. We also choose the specific labels for which we will train classifiers. Each label will have its own classifier.

3. **Retrieve Tweet Embeddings**: Load or generate tweet embeddings using a pre-trained model (e.g., GPT-4).

4. **Define Dataset, Model and Trainig loop**: Define the dataset and the neural network model according to Pytorch specifications which simplfy the training process dataloading and model training. Also create the functions that takes in all of this and trains the model.

5. **Train the Classifier**: Load the neural network classifier and train it using the selected hyperparameters.

##### 1. Load Cleaned Data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)

# Load the cleaned data
data = pd.read_csv('../data/cleaned_data.csv')

#Display the first 5 rows of the data and the shape of the data
display(data.shape)
display(data.head())

(613, 39)

,user,username,timestamp,post,permalink,reposts,likes,impressions,quotes,replies,bookmarks,value,report_file,date,time,relevance,anti RSF,pro RSF,anti SAF,pro SAF,"Pro peace,",anti peace,Pro War,anti war,pro civilian,anti civilians,not specified,no polarisation,Geopolticis,Sudanese,Not Sudanese,Sudanese N/A,Likely bot,Likely not a bot,Cannot be identified.,News,Not about Sudan,Annotation Confidence,code
0,☬ود نيـالا ☬,@Mo7amedWagi7,2023-04-15 15:29:44,@OmerElameen2 @drhzagalo لما يكون عندك حكومة و...,https://www.twitter.com/user/status/1647260928...,0,1,310,0,0,0,0.03,Report 1-1,2023-04-15,15:29:44,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,9.0,1647260928231055360
1,عربي21,@Arabi21News,2023-08-28 20:02:00,"الخارجية السودانية توضح لـ""عربي21"" حقيقة زيارة...",https://www.twitter.com/user/status/1696251807...,1,5,3402,1,1,0,3115.19,Report 3-2,2023-08-28,20:02:00,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,10.0,1696251807083758021
2,تحالف عاصفة الحزم,@AaSsiFfah2015,2023-04-15 22:52:29,#السودان \nنشرة المستشفيات:\nالحرب حتى الآن تس...,https://www.twitter.com/user/status/1647372349...,0,1,472,0,0,0,15.58,Report 1-3,2023-04-15,22:52:29,1,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0,8.0,1647372349585358848
3,H_Lamami,@HLamami2,2023-04-15 13:36:46,■اذا رايت الاخوان يحبون حميدتي ويطبلون له\nفاع...,https://www.twitter.com/user/status/1647232497...,0,0,72,0,0,0,4.04,Report 1-2,2023-04-15,13:36:46,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0,8.0,1647232497238028289
4,وفاء علي,@eali_wafa15327,2023-08-29 12:17:05,@taherabdulwhab @Mohmd_Elsiddig @saramubael م ...,https://www.twitter.com/user/status/1696497195...,0,4,30,0,1,0,0.05,Report 3-1,2023-08-29,12:17:05,1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,8.0,1696497195434881292


##### 2. Define Hyperparameters and Data Splits

In [2]:
import torch
import os

# Function to get the X and Y values from the dataframe. X is the tweets and Y is the labels
def get_xy(df, tweets_col, labels_cols):
    X = df[tweets_col].reset_index(drop=True)
    if labels_cols:
        Y = df[labels_cols].reset_index(drop=True)
    else:
        Y = df.drop(columns=['post']).reset_index(drop=True)
    codes = df['code'].reset_index(drop=True)
    return X, Y, codes

# Function to merge the pro and anti columns into a single column with values 0, 1 and 2
def merge_pro_anti(df, pro_col, anti_col):
    pro = df[pro_col].values.tolist()
    anti = df[anti_col].values.tolist()
    merged = []
    for i in range(len(pro)):
        if pro[i] == 1 and anti[i] == 0:
            merged.append(1)
        elif pro[i] == 0 and anti[i] == 1:
            merged.append(0)
        elif pro[i] == 0 and anti[i] == 0:
            merged.append(2)
        else:
            print(f' row {i} has both pro and anti')
            merged.append(2)
    return merged

# Set the accelerator to be used to one of 'cuda', 'mps' or 'cpu' based on availability
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

# The Classifiers can be binary (0, 1) or ternary (0, 1, 2).You can select which labels to merge into ternary and which to keep binary
data['RSF'] = merge_pro_anti(data, 'pro RSF', 'anti RSF')
data['SAF'] = merge_pro_anti(data, 'pro SAF', 'anti SAF')
# Select the Labels for which we want to train the classifier. One classifier will be trained for each label
labels = ['RSF', 'SAF', 'Pro peace,', 'anti peace',
          'Pro War', 'anti war', 'pro civilian', 'anti civilians', 
          'no polarisation', 'Geopolticis', 'Sudanese', 'Not Sudanese', 'anti RSF', 'pro RSF', 'anti SAF', 'pro SAF', ]
labels = labels[:4] # Remove this line, only here to speed up the training Using labels SAF and RSF (ternary) and Pro peace and anti peace (binary)


# Set the parameters for the training
test_size = 0.25 # Fraction of the data to be used for testing
random_state = 42 # Random state for reproducibility
n_splits = 5 # Number of splits for the cross validation
num_epochs = 5 # Number of epochs for training
batch_size = 256 # Batch size for training
hidden_dim = 1024 # Hidden dimension for the classifier
learning_rate = 1e-5 # Learning rate for the optimizer

# Get the X, Y and codes values. X is the tweets, Y is the labels and codes is the unique identifier for each tweet used later to retrieve its embeddings.
X, Y, codes = get_xy(data, tweets_col='post', labels_cols=labels)

# Display the shape of the X and Y values
display(X.shape, Y.shape)




(613,)

(613, 4)

##### 3. Retrieve Tweet Embeddings

In [3]:
from openai import OpenAI
from tqdm.auto import tqdm

# Function to get the OpenAI embeddings for the tweets in the data using the OpenAI API then save them to a parquet file
def save_openai_embs(df, api_key, model="text-embedding-3-large"):
    client = OpenAI(api_key=api_key)
    codes, tweets, large_embs = [], [], []
    for i,row in tqdm(df.iterrows(), total=df.shape[0]):
        tweet = row['post']
        code = row['code']
        response = client.embeddings.create(input=tweet,model=model)
        emb = response.data[0].embedding
        large_embs.append(emb)
        codes.append(str(code))
        tweets.append(tweet)
    embeddings_df = pd.DataFrame({'code': codes, 'tweet': tweets, 'embedding': large_embs})
    embeddings_df = embeddings_df.set_index('code', drop=True)
    os.makedirs('../embeddings', exist_ok=True)
    embeddings_df.to_parquet('../embeddings/labelled_embeddings.parquet')
    return embeddings_df

# Transoform the embeddings to a tensor based on the tweets in the data using their unique codes
def embeddings_tensor(codes, embs, device):
    tweets_embeddings = []
    for i in range(len(codes)):
        code = str(codes[i])
        row = embs.loc[code]
        tweets_embeddings.append(row['embedding'])
    tweets_embeddings = torch.tensor(tweets_embeddings, device=device, dtype=torch.float32)
    return tweets_embeddings


# Load the OpenAI embeddings if they exist
if os.path.exists('../embeddings/labelled_embeddings.parquet'):
    display('Loading Saved OpenAI Embeddings')
    openai_embs = pd.read_parquet('../embeddings/labelled_embeddings.parquet')
else:
    # Get the OpenAI embeddings for the tweets in the data using the OpenAI API then save them to a parquet file
    display('OpenAI Embeddings not found, using the API to get the embeddings')
    openai_embs = save_openai_embs(data, api_key='<sk-xxxxxx>') # Replace <sk-xxxxxx> with your OpenAI API key

# Get the embeddings tensor for the tweets in the data
embs = embeddings_tensor(codes, openai_embs, device)

# Get the embeddings dimension which will be used as the input size for the classifier
embeddings_dim = embs.shape[1]

# Display the shape of the embeddings tensor
display(embs.shape)

'Loading Saved OpenAI Embeddings'

torch.Size([613, 3072])

##### 4. Define Dataset, Model and Training Loop

In [4]:
import torch.nn as nn
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Function that takes a model, a dataloader, the number of classes and the device and returns the scores for the model
def score_model(model, dataloader, multi_calss, device):
    model.eval()
    ground_truth = []
    predictions = []
    with torch.no_grad():
        for x,y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)['probs']
            ground_truth.append(y)
            predictions.append(y_pred)
    ground_truth = torch.concat(ground_truth).detach().cpu()
    predictions = torch.concat(predictions).detach().cpu()
    gt = [yp.item() for yp in ground_truth]
    preds = [yp.argmax().item() for yp in predictions]
    acc = accuracy_score(gt, preds)
    f1 = f1_score(gt, preds, average='weighted')
    if multi_calss == 'raise':
        predictions = torch.tensor([yp.argmax().item() for yp in predictions])
    rocauc = roc_auc_score(ground_truth, predictions, multi_class=multi_calss)
    predictions = torch.where(predictions > 0.5, 1, 0).type(torch.float32)
    scores = {
        'rocauc': rocauc,
        'accuracy': acc,
        'f1': f1
    }
    return scores

# Create a Dataset class for the tweets and their labels
class TweetsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# Create a Classifier Neural Network class the defines the architecture of the classifier based on the input dimension, number of classes and hidden dimension
class TweetClassifer(nn.Module):
    def __init__(self, input_dim, num_classes, hidden_dim):
        super().__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.hidden_dim = hidden_dim
        self.fc1 = nn.Linear(self.input_dim, self.hidden_dim) # input dim -> 512
        self.fc2 = nn.Linear(self.hidden_dim, self.hidden_dim//2) # 512 -> 256
        self.fc3 = nn.Linear(self.hidden_dim//2, self.hidden_dim//4) # 256 -> 128
        self.fc4 = nn.Linear(self.hidden_dim//4, self.hidden_dim//8) # 128 -> 64
        self.fc5 = nn.Linear(self.hidden_dim//8, self.num_classes) # 64 -> num_classe
        self.softmax = nn.Softmax(dim=1)
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        logits = self.fc5(x)
        x = self.softmax(logits)
        return {'logits': logits, 'probs': x}
    
# Function to train the classifier using the dataloaders
def train(model, train_dataloader, val_dataloader, num_epochs, optimizer, loss_fn, num_classes,model_name, device):
    # Initialize the logs to store the training and validation metrics
    logs = {'train_loss': [], 'val_loss': [], 'train_rocauc':[], 'val_rocauc':[], 'train_acc':[], 'val_acc':[], 'train_f1':[], 'val_f1':[]}
    # Initialize the best accuracy to 0, used to save the best model
    acc_max = 0
    # Train the model for the number of epochs
    for epoch in tqdm(range(num_epochs)):
        # Set the model to training mode and iterate over the training dataloader updating the weights
        model.train()
        for x, y in train_dataloader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(x)['logits']
            loss = loss_fn(y_pred, y)
            loss.backward()
            optimizer.step()
        # Every 5 epochs, evaluate the model on the validation set and save the model if the accuracy is higher than the previous best
        if epoch%5 == 0:
            model.eval()
            with torch.no_grad():
                val_loss = 0
                for x, y in val_dataloader:
                    x = x.to(device)
                    y = y.to(device)
                    y_pred = model(x)['logits']
                    val_loss += loss_fn(y_pred, y)
            # Get the scores for the model on the training and validation sets
            multi_class = 'raise' if num_classes == 2 else 'ovo'
            train_scores = score_model(model, train_dataloader, multi_calss=multi_class, device=device)
            val_scores = score_model(model, val_dataloader, multi_calss=multi_class, device=device)
            train_rocauc, val_rocauc = train_scores['rocauc'], val_scores['rocauc']
            train_acc, val_acc = train_scores['accuracy'], val_scores['accuracy']
            train_f1, val_f1 = train_scores['f1'], val_scores['f1']
            # Save the model if the validation accuracy is higher than the previous best
            if val_acc > acc_max:
                acc_max = val_acc
                torch.save(model.state_dict(), f'../models/best_{model_name}.pth')
                print(f'>>> Best VAL ACC so far: {acc_max} at epoch {epoch} with train aucroc {train_rocauc} , train acc {train_acc} and val rocauc {val_rocauc} saved to ./models/best_{model_name}.pth')
            # Print the training and validation metrics
            print(f'Epoch: {epoch} -- Train Loss: {loss.item() :.4f} RocAuc = {train_rocauc*100 :.4f} Acc = {train_acc*100 :.4f}|| Val Loss: {val_loss.item() :.4f} RocAuc = {val_rocauc*100 :.4f} Acc = {val_acc*100 :.4f}')
            logs['train_loss'].append(loss.item())
            logs['val_loss'].append(val_loss.item())
            logs['train_rocauc'].append(train_rocauc)
            logs['val_rocauc'].append(val_rocauc)
            logs['train_acc'].append(train_acc)
            logs['val_acc'].append(val_acc)
            logs['train_f1'].append(train_f1)
            logs['val_f1'].append(val_f1)
    # Save the model after training
    torch.save(model.state_dict(), f'../models/lattest_{model_name}.pth')
    print(f'>>> Finished training {model_name} model and saved to ./models/lattest_{model_name}.pth, final metrics: train aucroc {train_rocauc} , train acc {train_acc} and val acc {val_acc}')
    return model, logs




##### 5. Train the Classifiers

In [5]:
import shutil
import numpy as np
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader
from sklearn.utils.class_weight import compute_class_weight


# Function to compute class weights to address class imbalance
def multi_class_weights(data, device):
    w = compute_class_weight(class_weight="balanced", classes=np.unique(data), y=data)
    w =torch.tensor(w, dtype=torch.float32).to(device=device)
    return(w)


os.makedirs('../models', exist_ok=True) # Create the models directory if it does not exist
# Creat the training Loop for Each Label
all_logs = {label: {} for label in labels}# Dictionary to store the logs for each label
labels_num_classes = [] # List to store the number of classes for each label (2 for binary and 3 for ternary). The number of classes will be used to create the classifier
for i,label in enumerate(labels):
    print(f'>>> Training model for {label}')
    x = embs.detach().cpu().numpy() # get x values from the embeddings tensor
    y = Y[label].to_numpy() # get y values from the current label
    num_classes = len(np.unique(y)) # get the number of classes for the current 
    labels_num_classes.append((label,num_classes))
    
    # Use Stratified KFold to split the data into n_splits folds
    skf = StratifiedKFold(n_splits=n_splits)

    # Loop through the splits and train the model
    for i, (train_index, test_index) in enumerate(skf.split(x, y)):
        # Generate the model name based on the label and the split
        model_name=f'clf_{label}_split_{i}'
        # Extract the train and test data as tensors in the device
        x_train, y_train = x[train_index], y[train_index]
        x_test, y_test = x[test_index], y[test_index]
        x_train= torch.tensor(x_train, dtype=torch.float32, device=device)
        y_train= torch.tensor(y_train, dtype=torch.float32, device=device)
        x_test = torch.tensor(x_test , dtype=torch.float32, device=device)
        y_test = torch.tensor(y_test , dtype=torch.float32, device=device)

        # Create the datasets and dataloaders
        train_ds = TweetsDataset(x_train, y_train)
        test_ds = TweetsDataset(x_test, y_test)
        train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
        test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

        # Create the model, optimizer and loss function
        model = TweetClassifer(embeddings_dim, num_classes, hidden_dim=hidden_dim).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        loss_fn = torch.nn.CrossEntropyLoss(weight=multi_class_weights(data=y, device=device))

        # Train the model
        model, logs = train(model, train_dataloader, test_dataloader, num_epochs, optimizer, loss_fn, num_classes, model_name, device)
        all_logs[label][i] = logs

# Save the logs using torch.save
os.makedirs('../logs', exist_ok=True)
torch.save(all_logs, '../logs/stratified_logs.pth')

#Finally, rename the models with the best validation F1 score across all splits for each label
for label in all_logs:
    f1 = -float('inf')
    best_split = None
    for split in all_logs[label]:
        logs = all_logs[label][split]
        if logs['val_f1'][-1] > f1:
            f1 = logs['val_f1'][-1]
            best_split = split
    best_split_model_path = f'../models/best_clf_{label}_split_{best_split}.pth'
    new_model_path = f'../models/clf_{label}.pth'
    shutil.copy(best_split_model_path, new_model_path)
    
        



>>> Training model for RSF


  0%|          | 0/5 [00:00<?, ?it/s]

>>> Best VAL ACC so far: 0.6422764227642277 at epoch 0 with train aucroc 0.6252563344178251 , train acc 0.6428571428571429 and val rocauc 0.4916798523206751 saved to ./models/best_clf_RSF_split_0.pth
Epoch: 0 -- Train Loss: 1.1010 RocAuc = 62.5256 Acc = 64.2857|| Val Loss: 1.0993 RocAuc = 49.1680 Acc = 64.2276
>>> Finished training clf_RSF_split_0 model and saved to ./models/lattest_clf_RSF_split_0.pth, final metrics: train aucroc 0.6252563344178251 , train acc 0.6428571428571429 and val acc 0.6422764227642277


  0%|          | 0/5 [00:00<?, ?it/s]

>>> Best VAL ACC so far: 0.032520325203252036 at epoch 0 with train aucroc 0.6471392421702981 , train acc 0.02857142857142857 and val rocauc 0.6409150843881857 saved to ./models/best_clf_RSF_split_1.pth
Epoch: 0 -- Train Loss: 1.0939 RocAuc = 64.7139 Acc = 2.8571|| Val Loss: 1.0974 RocAuc = 64.0915 Acc = 3.2520
>>> Finished training clf_RSF_split_1 model and saved to ./models/lattest_clf_RSF_split_1.pth, final metrics: train aucroc 0.6471392421702981 , train acc 0.02857142857142857 and val acc 0.032520325203252036


...

>>> Best VAL ACC so far: 0.926829268292683 at epoch 0 with train aucroc 0.5 , train acc 0.9346938775510204 and val rocauc 0.5 saved to ./models/best_clf_anti peace_split_2.pth
Epoch: 0 -- Train Loss: 0.6929 RocAuc = 50.0000 Acc = 93.4694|| Val Loss: 0.6931 RocAuc = 50.0000 Acc = 92.6829
>>> Finished training clf_anti peace_split_2 model and saved to ./models/lattest_clf_anti peace_split_2.pth, final metrics: train aucroc 0.5 , train acc 0.9346938775510204 and val acc 0.926829268292683


  0%|          | 0/5 [00:00<?, ?it/s]

>>> Best VAL ACC so far: 0.06557377049180328 at epoch 0 with train aucroc 0.5 , train acc 0.06720977596741344 and val rocauc 0.5 saved to ./models/best_clf_anti peace_split_3.pth
Epoch: 0 -- Train Loss: 0.7120 RocAuc = 50.0000 Acc = 6.7210|| Val Loss: 0.6950 RocAuc = 50.0000 Acc = 6.5574
>>> Finished training clf_anti peace_split_3 model and saved to ./models/lattest_clf_anti peace_split_3.pth, final metrics: train aucroc 0.5 , train acc 0.06720977596741344 and val acc 0.06557377049180328


  0%|          | 0/5 [00:00<?, ?it/s]

>>> Best VAL ACC so far: 0.9344262295081968 at epoch 0 with train aucroc 0.5 , train acc 0.9327902240325866 and val rocauc 0.5 saved to ./models/best_clf_anti peace_split_4.pth
Epoch: 0 -- Train Loss: 0.6890 RocAuc = 50.0000 Acc = 93.2790|| Val Loss: 0.6932 RocAuc = 50.0000 Acc = 93.4426
>>> Finished training clf_anti peace_split_4 model and saved to ./models/lattest_clf_anti peace_split_4.pth, final metrics: train aucroc 0.5 , train acc 0.9327902240325866 and val acc 0.9344262295081968
